In [184]:
# pip install librosa

In [185]:
'''
Please install this specific version of resampy for librosa to work without errors.
'''

'\nPlease install this specific version of resampy for librosa to work without errors.\n'

In [186]:
# pip install resampy==0.3.1

In [187]:
import soundfile
import os
import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import librosa
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report
import warnings; warnings.filterwarnings('ignore')

In [188]:
emotions ={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

### Data for binary classification


In [189]:
def load_extract_features(data_path):

    '''
    load_extract_features() is a function that is used to load all the audio files one at a time, compute their features and return the features as well as the target values.

    There are around 8-10 audio files which are corrupted. We hardcode zero values for such files in order to maintain consistency.

    ['calm', 'happy'] emotion data is categorized into 'positive' and  ['angry', 'fearful'] into 'negative'

    Returns:
    1. Features
    2. Binary Target Values
    '''
    final_features,target_emotions, binary_label = [],[], []
    count = 0
    
    for i in glob.glob(data_path + "/Actor_*/*.wav"): #Loop to read every file.
        
        name = os.path.basename(i)
        #We split the name of the file to understand the emotion associated with the file.
        split = name.split("-")
        #We know that the third identifier is associated with the emotion of the audio file. Hence, we use [2] as it represents the third identifier.
        emotion = emotions[split[2]]

        #Below is the code to categorize the emotions into two classes to make this a binary problem.
        if emotion in ['calm', 'happy']:
            binary_label.append(0)
        elif emotion in ['angry', 'fearful']:
            binary_label.append(1)
        else:
            continue
        
        with soundfile.SoundFile(i) as audio:
            waveform = audio.read(dtype="float32")
            sr = audio.samplerate
            
            #Below is the code to extract the Mel spectrogram features
            #128 is the standard for machine learning applications using Mel spectrograms
            m_feature = librosa.feature.melspectrogram(y=waveform, sr=sr, n_mels=128, fmax=sr / 2.0).T
            melspectrogram = np.mean(m_feature,axis=0)
            if melspectrogram.shape != (128,):
                melspectrogram = np.zeros(128)
            
            #Below is the code to extract the chromagram features
            stft_wave = librosa.stft(waveform)
            stft = np.abs(stft_wave)
            c_feature = librosa.feature.chroma_stft(S=stft, sr=sr).T
            chromagram = np.mean(c_feature,axis=0)
            
            #12 is the number of pitch classes
            if chromagram.shape != (12,):
                chromagram = np.zeros(12)
                
            features=np.array([])
            features=np.hstack((chromagram, melspectrogram))
        
            final_features.append(features)
            target_emotions.append(emotion)
            
            count += 1
            if count % 100 == 0:
                print("Processed Audio File Number: ", count)
    
    #We return the features and the binary target values.
    return np.array(final_features), np.array(binary_label)

In [190]:
#Please change the path below to the path of the folder saved in your computer.
# data_path = './Audio_Speech_Actors_01-24'
# X, binary_label = load_extract_features(data_path)
# print(X.shape)
# print(binary_label.shape)


In [191]:
# np.savetxt('matrix.txt', X, delimiter = ',')  
# np.savetxt('binary_label.txt',binary_label,delimiter = ',')

In [192]:
X = np.loadtxt(open("matrix.txt","rb"), delimiter=",", skiprows=0)
binary_label = np.loadtxt(open("binary_label.txt","rb"), delimiter=",", skiprows=0)

print(X.shape) #should be (768,140)
print(binary_label.shape) # should be (768,)

(768, 140)
(768,)


In [193]:
X_train, X_test, Y_train, Y_test = train_test_split(X, binary_label, test_size=0.3, random_state=0)

In [194]:
def compute_cost(W, X, Y, l):
    # calculate hinge loss
    n_s = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss =  (np.sum(distances) / n_s)
    # calculate cost
    cost = l / 2 * np.dot(W, W) + hinge_loss 
    
    return cost

In [195]:
def calculate_gradient(W, X, Y,l):

    distance = 1 - (Y * np.dot(X, W))
    dw = np.zeros(len(W))

    for ind, d in enumerate(distance):
        if (ind == W.shape[0]):
            if max(0, d) != 0:
                di =  - ( Y[ind] * X[ind])
        else:
            if max(0, d) == 0:
                di = l*W
            else:
                di = l*W - ( Y[ind] * X[ind])
        dw += di

    dw = dw/len(Y)  # average

    return dw

In [196]:
def svm(X, Y, step, l,max_iter):
    X_bia = np.c_[X, np.ones(X.shape[0])]   # Pad 1's for the bias term
    new_Y = np.ones((Y.shape[0]))

    for i in range (Y.shape[0]):
        if Y[i] == 0:
            new_Y[i] = -1

    w = np.zeros(X_bia.shape[1])
    count = 0
    cost = compute_cost(w, X_bia, new_Y, l)
    for i in range(max_iter):
        grad = calculate_gradient(w, X_bia, new_Y, l)
        new_w = w - step * grad
        new_cost = compute_cost(new_w, X_bia, new_Y, l)
        print(cost)
        if (new_cost<cost):
            w = new_w
            cost = new_cost
            count= count +1
        else:
            break
    print(count)
    return w

In [197]:
def svm_accuracy(W,X,Y):

    X_bia = np.c_[X, np.ones(X.shape[0])]   # Pad 1's for the bias term
    new_Y = np.ones((Y.shape[0]))

    for i in range (Y.shape[0]):
        if Y[i] == 0:
            new_Y[i] = -1

    count = 0

    for i in range(X.shape[0]):
        yp = np.sign(np.dot(W, X_bia[i])) #model
        if(yp == new_Y[i]):
            count = count +1

    return count / Y.shape[0]

In [198]:
l = 10  # lambda
w_result = svm(X_train, Y_train, 0.001, l,3000)
print(svm_accuracy(w_result, X_train, Y_train))
print(svm_accuracy(w_result, X_test, Y_test))

1.0
0.9890221031827218
0.9782624651433336
0.9677167485494828
0.9579615519118195
0.9491381273528314
0.9410869845226995
0.9342676213580026
0.9289865942811809
0.924733766327447
0.9207930376285539
0.9176052182989474
0.9150061227342403
0.9127306958441642
0.9106339820967041
0.9086672371618023
0.9072234402378486
0.9060034917115621
0.9049501609175247
0.9039753059258787
0.9031442511495736
0.9024132607423988
0.901755469192951
0.9011911152917711
0.9007385135045076
0.9003294903667488
0.8999286041434071
0.8995561516487622
0.8992163806559244
0.8988852040562179
0.8986035718840285
0.8983714387965507
0.8981439244945001
0.8979209371555357
0.8977023867819813
0.8974881851645755
0.8972842691892843
0.8971042976060529
0.8969279074340727
0.8967550274693098
0.8965855879227244
0.8964230887868535
0.8962687132650586
0.8961174099137946
0.8959691176523545
0.8958248598294228
0.8956825395932574
0.8955429023931997
0.8954060443280517
0.8952719101405698
0.895140445671641
0.8950115978384636
0.8948853146131645
0.894761545

In [199]:
def pca(X, k):
    mean = np.mean(X, axis = 0)   # the mean value of X
    std = np.std(X, axis=0)       # the standard deviation along axis 0
    X_std = (X - mean) / std      # standardized data
    cov_mat = np.cov(X_std.T)     # calculate the covariance matrix

    values, vectors = np.linalg.eig(cov_mat)   # Eigendecomposition of covariance matrix

    # Make a list of (eigenvalue, eigenvector) tuples
    eig_pairs = [(np.abs(values[i]), vectors[i,:]) for i in range(len(values))]

    # Sort the tuples from the highest to the lowest based on eigenvalues magnitude
    eig_pairs.sort(key=lambda x: x[0], reverse=True)

    # Store the soted eigenvalues and their corresponding eigenvectors
    vecs_sorted = np.array([x[1] for x in eig_pairs])

    W = vecs_sorted[:k, :]   # Projection matrix with the top k eigenvectors selected

    return W

In [200]:
W = pca(X_train, 40)
W.shape

(40, 140)

In [201]:
X_proj = X_train.dot(W.T)             # transform X via W to obtain a k-dimensional feature subspace.
w = svm(X_proj, Y_train, 0.0001, l)
print(svm_accuracy(w, X_proj, Y_train))

TypeError: svm() missing 1 required positional argument: 'max_iter'

In [ ]:
X_proj = X_test.dot(W.T)       # transform X via W to obtain a k-dimensional feature subspace.
print(svm_accuracy(w, X_proj, Y_test))

0.5714285714285714
